In [1]:
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from hydromt_sfincs import SfincsModel
from matplotlib import animation

Load model.

In [2]:
# select the example model
sfincs_root = Path(r"model/base_garissa")  # (relative) path to sfincs root
model = SfincsModel(sfincs_root, mode="r")

Read results:

In [3]:
model.read_results()

print(model.results.keys())

dict_keys(['inp', 'msk', 'zb', 'zs', 'zsmax', 'total_runtime', 'average_dt'])


### Creating an animation

This code is used to create animation of the depth.

In [5]:
zs = model.results["zs"]
zsmax = zs.max(dim='time')
 
bed_level =  model.results["zb"].copy()
water_level_max = zsmax - bed_level

# Create quick video
water_level_max = model.results["zs"].copy()

step = 1  # one frame every <step> dtout
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}

def update_plot(i, water_level_max, cax_h):
    water_level_max = water_level_max.isel(time=i)
    t = water_level_max.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    ax.set_title(f"SFINCS water depth {t}")
    cax_h.set_array(water_level_max.values.ravel())

fig, ax = model.plot_basemap(
    fn_out=None, variable="", bmap="sat", plot_bounds=False, figsize=(7, 7)
)
cax_h = water_level_max.isel(time=0).plot(
    x="x", y="y", ax=ax, 
    vmin=0.0, vmax=6.0,
    cmap=plt.cm.viridis,
    alpha=0.75, cbar_kwargs=cbar_kwargs)

plt.close()  # to prevent double plot

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, water_level_max.time.size, step),
    interval=250,  # ms between frames
    fargs=(water_level_max, cax_h,),
)

# # to show in notebook:
from IPython.display import HTML

HTML(ani.to_html5_video())

# from IPython.display import Video

# # Save animation as a video file
# ani.save('animation.mp4', fps=30,)

# # Display the saved video
# Video("animation.mp4", width=600)

RuntimeError: Requested MovieWriter (ffmpeg) not available

### Determining the maximum flood depth
This code is used to determine the maximum flood depth over the simulation.

In [ ]:
zs = model.results["zs"]
zsmax = zs.max(dim='time')
 
bed_level =  model.results["zb"].copy()
water_level_max = zsmax - bed_level
 
water_level_max
 
# hmax = utils.downscale_floodmap(
#     zsmax=water_level_max,
#     dep=dep, #Use subgrid
#     hmin=hmin,
#     # gdf_mask=gdf_osm,
#     # floodmap_fn=join(sfincs_root, "floodmap.tif") # uncomment to save to <mod.root>/floodmap.tif
# )
 
 
 
fig, ax = model.plot_basemap(
    fn_out=None,
    variable="", # no variable to plot, only basemap
    plot_bounds=False,
    bmap="sat",
    figsize=(11, 7),
    zoomlevel = 12
)
 
 
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
cax_fld = water_level_max.plot(
    x="x", y="y",
    ax=ax,
    vmin=0.0, vmax=6.0,
    cmap=plt.cm.viridis,
    cbar_kwargs=cbar_kwargs,
)
 
ax.set_title(f"SFINCS maximum water depth")
#plt.savefig(join(scenario_dir, 'hmax.png'), dpi=225, bbox_inches="tight")
plt.show() 